<a href="https://colab.research.google.com/github/MeenTers/Project/blob/main/Train_Model_Car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation Package

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
from torch import nn, load, utils
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms, models
from torchvision.models.vgg import model_urls
from os import path, listdir
model_urls['vgg19'] = model_urls['vgg19'].replace('https://', 'http://')

In [3]:
from scipy.spatial import distance_matrix
from scipy.spatial import distance
import pandas as pd

In [4]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2

# Create Functions

## VGG-19 Pretrain with ImageNet

In [5]:
class VGG:
	def __init__(self):
		model = models.vgg19(pretrained=True, progress=True)
		model.classifier = nn.Sequential(*list(model.classifier.children())[:3])
		self.model = model.cuda().eval()

	def __call__(self, x):
		return self.model(x)

In [6]:
vgg = VGG()

Downloading: "http://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

## Features Extraction Function

In [7]:
def get_features(model, loader):
    features = []
    with torch.no_grad():
        for batch, _ in tqdm(loader):
            if torch.cuda.is_available():
                batch = batch.cuda()
            b_features = model(batch).detach().cpu().numpy()
            for f in b_features:
                features.append(f)

    return features

## Loading the data Function

In [10]:
def get_dataset(images_path):
  transform = transforms.Compose([
    transforms.Resize(size=32),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  dataset = datasets.ImageFolder(images_path, transform=transform)
  loader = utils.data.DataLoader(dataset, batch_size=100, shuffle=False, num_workers=1, pin_memory=True)
  return loader

## Compute Distance Function

In [9]:
def get_euclidean(base_car,diff_view):
    A = []
    for i in range(len(diff_view)):
        x = distance.euclidean(base_car[i],diff_view[i])
        A.append(x)
    return(A)

# Front Views

## Little Damage

In [12]:
# Loading Data
little_base_front_loader = get_dataset(r'/content/drive/MyDrive/train_model/Base/Front')
little_dmg_front_loader  = get_dataset(r'/content/drive/MyDrive/train_model/Damage/Front/Little Damage')

In [13]:
#Extract Features
little_base_front_feat = get_features(vgg,little_base_front_loader)
little_dmg_front_feat  = get_features(vgg,little_dmg_front_loader)

100%|██████████| 1/1 [00:03<00:00,  3.70s/it]


In [14]:
little_front_dist = get_euclidean(little_base_front_feat,little_dmg_front_feat)
little_front_dist

[5.304965019226074,
 2.2764980792999268,
 6.874874591827393,
 10.913042068481445,
 14.467225074768066,
 9.235076904296875,
 18.01740264892578]

## Damage

In [15]:
# Loading Data
base_front_loader = get_dataset(r'/content/drive/MyDrive/train_model/Base/Front')
dmg_front_loader  = get_dataset(r'/content/drive/MyDrive/train_model/Damage/Front/Damage')

In [16]:
#Extract Features
base_front_feat = get_features(vgg,base_front_loader)
dmg_front_feat  = get_features(vgg,dmg_front_loader)

100%|██████████| 1/1 [00:03<00:00,  3.55s/it]


In [17]:
front_dist = get_euclidean(base_front_feat,dmg_front_feat)
front_dist

[27.097463607788086,
 9.43012523651123,
 36.0161247253418,
 30.508304595947266,
 15.178224563598633,
 11.249543190002441,
 23.93056869506836]

## Most Damage

In [18]:
# Loading Data
base_front_loader = get_dataset(r'/content/drive/MyDrive/train_model/Base/Front')
most_dmg_front_loader  = get_dataset(r'/content/drive/MyDrive/train_model/Damage/Front/Most Damage')

In [19]:
#Extract Features
base_front_feat = get_features(vgg,base_front_loader)
most_dmg_front_feat  = get_features(vgg,most_dmg_front_loader)

100%|██████████| 1/1 [00:03<00:00,  3.58s/it]


In [20]:
most_front_dist = get_euclidean(base_front_feat,most_dmg_front_feat)
most_front_dist

[68.19573211669922,
 57.24534225463867,
 49.86717987060547,
 46.62443923950195,
 28.867015838623047,
 13.375458717346191,
 49.08580780029297]

# Left View

## Little Damage

In [21]:
# Loading Data
base_left_loader = get_dataset(r'/content/drive/MyDrive/train_model/Base/Left')
little_dmg_left_loader  = get_dataset(r'/content/drive/MyDrive/train_model/Damage/Left/Little Damage')

In [22]:
#Extract Features
base_left_feat = get_features(vgg,base_left_loader)
little_dmg_left_feat  = get_features(vgg,little_dmg_left_loader)

100%|██████████| 1/1 [00:03<00:00,  3.81s/it]


In [23]:
little_left_dist = get_euclidean(base_left_feat,little_dmg_left_feat)
little_left_dist

[1.236898422241211,
 0.7314380407333374,
 0.0,
 2.2292003631591797,
 0.0,
 0.0,
 0.15286785364151]

## Damage

In [24]:
# Loading Data
dmg_left_loader  = get_dataset(r'/content/drive/MyDrive/train_model/Damage/Left/Damage')

In [25]:
#Extract Features
dmg_left_feat  = get_features(vgg,dmg_left_loader)

100%|██████████| 1/1 [00:03<00:00,  3.49s/it]


In [26]:
left_dist = get_euclidean(base_left_feat,dmg_left_feat)
left_dist

[19.150352478027344,
 36.80379867553711,
 24.716638565063477,
 27.39190673828125,
 0.0,
 16.895296096801758,
 20.503732681274414]

## Most Damage

In [27]:
# Loading Data
most_dmg_left_loader  = get_dataset(r'/content/drive/MyDrive/train_model/Damage/Left/Most Damage')

In [28]:
#Extract Features
most_dmg_left_feat  = get_features(vgg,most_dmg_left_loader)

100%|██████████| 1/1 [00:03<00:00,  3.39s/it]


In [29]:
most_left_dist = get_euclidean(base_left_feat,most_dmg_left_feat)
most_left_dist

[38.60552215576172,
 42.035465240478516,
 53.78656005859375,
 32.845802307128906,
 25.846031188964844,
 23.10930633544922,
 34.61317443847656]

# Back

## Little Damage

In [30]:
# Loading Data
base_back_loader = get_dataset(r'/content/drive/MyDrive/train_model/Base/Back')
little_dmg_back_loader  = get_dataset(r'/content/drive/MyDrive/train_model/Damage/Back/Little Damage')

In [31]:
#Extract Features
base_back_feat = get_features(vgg,base_back_loader)
little_dmg_back_feat  = get_features(vgg,little_dmg_back_loader)

100%|██████████| 1/1 [00:03<00:00,  3.69s/it]


In [32]:
little_back_dist = get_euclidean(base_back_feat,little_dmg_back_feat)
little_back_dist

[8.860122680664062,
 2.142881393432617,
 2.4771201610565186,
 27.078006744384766,
 1.4209057092666626,
 10.339546203613281,
 10.107410430908203]

# Damage

In [33]:
# Loading Data
dmg_back_loader  = get_dataset(r'/content/drive/MyDrive/train_model/Damage/Back/Damage')

In [34]:
#Extract Features
dmg_back_feat  = get_features(vgg,dmg_back_loader)

100%|██████████| 1/1 [00:03<00:00,  3.43s/it]


In [35]:
back_dist = get_euclidean(base_back_feat,dmg_back_feat)
back_dist

[8.79023551940918,
 11.450382232666016,
 2.6085054874420166,
 33.32075881958008,
 7.946778774261475,
 11.522150039672852,
 18.107521057128906]

## Most Damage

In [36]:
# Loading Data
most_dmg_back_loader  = get_dataset(r'/content/drive/MyDrive/train_model/Damage/Back/Most Damage')

In [37]:
#Extract Features
most_dmg_back_feat  = get_features(vgg,most_dmg_back_loader)

100%|██████████| 1/1 [00:03<00:00,  3.60s/it]


In [38]:
most_back_dist = get_euclidean(base_back_feat,most_dmg_back_feat)
most_back_dist

[46.44422149658203,
 72.7226791381836,
 9.393021583557129,
 51.96744155883789,
 45.819881439208984,
 37.23289489746094,
 79.67497253417969]

# Right

## Little Damage

In [39]:
# Loading Data
base_right_loader = get_dataset(r'/content/drive/MyDrive/train_model/Base/Right')
little_dmg_right_loader  = get_dataset(r'/content/drive/MyDrive/train_model/Damage/Right/Little Damage')

In [40]:
#Extract Features
base_right_feat = get_features(vgg,base_right_loader)
little_dmg_right_feat  = get_features(vgg,little_dmg_right_loader)

100%|██████████| 1/1 [00:04<00:00,  4.04s/it]


In [41]:
little_right_dist = get_euclidean(base_right_feat,little_dmg_right_feat)
little_right_dist

[1.482006311416626,
 12.650895118713379,
 0.0,
 0.14384199678897858,
 12.013653755187988,
 0.27887779474258423,
 0.35788145661354065]

## Damage

In [42]:
# Loading Data
dmg_right_loader  = get_dataset(r'/content/drive/MyDrive/train_model/Damage/Right/Damage')

In [43]:
#Extract Features
dmg_right_feat  = get_features(vgg,dmg_right_loader)

100%|██████████| 1/1 [00:03<00:00,  3.43s/it]


In [44]:
right_dist = get_euclidean(base_right_feat,dmg_right_feat)
right_dist

[13.210777282714844,
 32.16436767578125,
 14.869745254516602,
 21.870052337646484,
 49.970550537109375,
 24.162689208984375,
 17.059917449951172]

## Most Damage

In [45]:
# Loading Data
most_dmg_right_loader  = get_dataset(r'/content/drive/MyDrive/train_model/Damage/Right/Most Damage')

In [46]:
#Extract Features
most_dmg_right_feat  = get_features(vgg,most_dmg_right_loader)

100%|██████████| 1/1 [00:03<00:00,  3.50s/it]


In [47]:
most_right_dist = get_euclidean(base_right_feat,most_dmg_right_feat)
most_right_dist

[27.740497589111328,
 29.7847843170166,
 36.266639709472656,
 25.08666229248047,
 45.91991424560547,
 24.597291946411133,
 44.16800308227539]